# Objectives

- To verify imputed cost for Rx encounters


In [1]:
## Import required libraries 

import snowflake.connector
import getpass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 400)
import plotly.graph_objects as go
pd.set_option("display.max_columns", 100)

In [2]:
import sys
sys.path.append('/home/jovyan/credentials')
import jupytertoolz_credentials as jt

In [3]:
## Snowflake tables 
ROLE = 'ANALYST'
WAREHOUSE = 'XLARGE_WH'
DATABASE = 'SANDBOX_KOMODO'  
SCHEMA = 'PROJECT_CURRENCY'

connection = snowflake.connector.connect(user='YWEI', password=getpass.getpass(), account='komodohealth')
connection.cursor().execute('USE ROLE ' + ROLE)
connection.cursor().execute('USE WAREHOUSE ' + WAREHOUSE)

def read_sql(sql, connection = connection):
    df = pd.read_sql(sql, connection)
    return df

def execute_sql(sql, connection = connection):
    #pd.io.sql.execute(sql, connection)
    connection.cursor().execute(sql)
    
execute_sql(f"USE ROLE ANALYST")
execute_sql(f"USE database {DATABASE}")
execute_sql(f"USE SCHEMA {SCHEMA}")

········


### Check List

1. NDC Validation 
2. distribution of days_supply, quantity, unit_price, cost
3. distribution of cost by year
4. frequency of imputation methods used
5. imputed vs actual
6. identify ndcs with cost outliers for further investigations 
7. route?
8. unit price comparison?

In [4]:
## Table reference
rx_version = '20220511'
rx_enc = f"MAP_ENCOUNTERS.RX_ENCOUNTERS_{rx_version}.RX_ENCOUNTER_LS_GA"
rx_claim = f"MAP_CERTIFIED_CLAIMS.PUBLIC.VIEW_CERTIFIED_CLAIMS_PHARMACY_WITH_JENNER"

dm_version = '20220513'
dm_file = f"MAP_VOCABULARY.RXNORM_{dm_version}.DRUG_MASTER_ACTIVE_AND_HISTORICAL"

asp_price = 'ASP_PRICE_2022JAN'
asp_xwalk = 'ASP_NDC_HCPCS_CROSSWALK_2022JAN'
asp_noc_price = 'ASP_NOC_PRICE_2022JAN'
asp_noc_xwalk = 'ASP_NOC_NDC_HCPCS_CROSSWALK_2022JAN'
nadac_xwalk = 'NADAC_20220322'

cpi_index = "CPI_INDEX_MEDICALSERVICES_20220627"
    
sample = 1
seed = 1234

In [5]:
# define output table names - easier to rerun intermediate steps
input_source = 'claim'
# input_source = 'encounter'

if input_source == 'encounter':
    prefix = f"RX_ENCOUNTERS_{rx_version}"
elif input_source == 'claim':
    prefix = 'RX_CERTIFIED_CLAIMS'
    
# get all claims, filter to PAID and cleaning
rx_enc_step1 = f"{prefix}_STEP1"

# quantity imputation
days_supply_quantity_dist = f"{prefix}_DAYS_SUPPLY_QUANTITY_DIST"
days_supply_quantity_guess = f"{prefix}_DAYS_SUPPLY_QUANTITY_GUESS"
quantity_guess = f"{prefix}_QUANTITY_GUESS"
rx_enc_step2 = f"{prefix}_STEP2"

# unit price imputation
asp_per_unit_price = f"{prefix}_ASP_PER_UNIT_PRICE"
nadac_per_unit_price = f"{prefix}_NADAC_PER_UNIT_PRICE"
median_unit_price = f"{prefix}_MEDIAN_UNIT_PRICE"
unit_price = f"{prefix}_UNIT_PRICE"

# final imputation
cost_per_days_supply = f"{prefix}_COST_PER_DAYS_SUPPLY"
rx_enc_impute = f"{prefix}_IMPUTED"

# define different variable names for different input tables
if input_source == 'encounter':
    input_table = rx_enc
    patient_key = 'upk_key2'
    claim_key = 'encounter_key'
    claim_date = 'claim_date'
    ndc_var = 'ndc'
    source_var = 'sources'
elif input_source == 'claim':
    input_table = rx_claim
    patient_key = 'upk_key2'
    claim_key = 'claim_id'
    claim_date = 'date_of_service'
    ndc_var = 'ndc11'
    source_var = 'source'


In [13]:
%%time
# create 1% sample for key datasets
# run analysis using 1p tables
rx_enc_step1_1p = rx_enc_step1 + "_1p"
rx_enc_step2_1p = rx_enc_step2 + "_1p"
rx_enc_impute_1p = rx_enc_impute + "_1p"

sql = f"""
    create or replace table {rx_enc_step1}_1p as select * from {rx_enc_step1} sample (1) seed (1234);
"""
execute_sql(sql)

sql = f"""
    create or replace table {rx_enc_step2}_1p as select * from {rx_enc_step2}
    where {claim_key} in (select {claim_key} from {rx_enc_step1}_1p);
"""
execute_sql(sql)

sql = f"""
    create or replace table {rx_enc_impute}_1p as select * from {rx_enc_impute}
    where {claim_key} in (select {claim_key} from {rx_enc_step1}_1p);
"""
execute_sql(sql)

CPU times: user 357 ms, sys: 13.7 ms, total: 371 ms
Wall time: 19min 52s


In [7]:
%%time
# NDC validation against DM, NADAC, ASP
sql = f"""
    with tmp as (
    select {ndc_var},
        {ndc_var} in (select distinct ndc from {dm_file}) as in_dm,
        {ndc_var} in (select distinct ndc2 from {asp_xwalk}) as in_asp,
        {ndc_var} in (select distinct NDC_OR_ALTERNATE_ID from {asp_noc_xwalk}) as in_asp_noc,
        {ndc_var} in (select distinct ndc from {nadac_xwalk}) as in_nadac
    from {rx_enc_step1}_1p
    )
    select 
        count(*) as claim, 
        count_if({ndc_var} is null or length({ndc_var}) < 11 or left({ndc_var}, 5) = '00000' or left({ndc_var}, 5) = '99999') as claim_null, 
        count(distinct {ndc_var}) as code, 
        count_if(in_dm) as claim_dm,
        count(distinct (case when in_dm then {ndc_var} end)) as code_dm,
        count_if(in_asp) as claim_asp,
        count(distinct (case when in_asp then {ndc_var} end)) as code_asp,
        count_if(in_asp_noc) as claim_asp_noc,
        count(distinct (case when in_asp_noc then {ndc_var} end)) as code_asp_noc,
        count_if(in_nadac) as claim_nadac,
        count(distinct (case when in_nadac then {ndc_var} end)) as code_nadac,
        count_if(in_dm or in_nadac or in_asp or in_asp_noc) as claim_any,
        count(distinct (case when in_dm or in_nadac or in_asp or in_asp_noc then {ndc_var} end)) as code_any
    from tmp
    ;
"""
df_ndc = read_sql(sql)
display(df_ndc)

,CLAIM,CLAIM_NULL,CODE,CLAIM_DM,CODE_DM,CLAIM_ASP,CODE_ASP,CLAIM_ASP_NOC,CODE_ASP_NOC,CLAIM_NADAC,CODE_NADAC,CLAIM_ANY,CODE_ANY
0,314533553,590275,141129,305521537,86320,6323781,4020,14125,216,245247403,24143,305681532,86554


CPU times: user 116 ms, sys: 8.18 ms, total: 125 ms
Wall time: 1min 42s


In [8]:
def get_dist_cts(var, table, filter = 'TRUE', filter_print = None, group_by_list = None):
    if filter_print is None:
        filter_print = filter
    group_by_str = ''
    order_by_str = ''
    group_by_select_str = ''
    if group_by_list is not None:
        group_by = ",".join(group_by_list)
        group_by_select_str = group_by + ','
        group_by_str = 'group by ' + group_by
        order_by_str = 'order by ' + group_by
    return f"""
        select 
            {group_by_select_str}
            '{var}' as var, 
            '{filter_print}' as filter,
            count(*) as n,
            count_if({var} is null) as n_null,
            count_if({var} < 0) as n_neg,
            count_if({var} = 0) as n_zero,
            count_if({var} > 0) as n_pos,
            avg({var}) as mean,
            variance({var}) as variance,
            min({var}) as min,
            PERCENTILE_CONT(0.01) WITHIN GROUP (ORDER BY {var}) as p_01,
            PERCENTILE_CONT(0.05) WITHIN GROUP (ORDER BY {var}) as p_05,
            PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY {var}) as p_10,
            PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY {var}) as p_25,
            PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY {var}) as median,
            PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY {var}) as p_75,
            PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY {var}) as p_90,
            PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY {var}) as p_95,
            PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY {var}) as p_99,
            max({var}) as max
        from {table}
        where {filter}
        {group_by_str}
        {order_by_str}
    """

In [14]:
%%time
# check distribution of days_supply, quantity, unit_cost and cost

days_supply = get_dist_cts("days_supply", rx_enc_step1_1p)
days_supply_pos = get_dist_cts("days_supply", rx_enc_step1_1p, filter = "days_supply > 0")
days_supply_clean = get_dist_cts("days_supply_clean", rx_enc_step2_1p)

quantity_dispensed = get_dist_cts("quantity_dispensed", rx_enc_step1_1p)
quantity_dispensed_pos = get_dist_cts("quantity_dispensed", rx_enc_step1_1p, filter = "quantity_dispensed > 0")
quantity_final = get_dist_cts("quantity_final", rx_enc_step2_1p)

plan_pay_actual = get_dist_cts("plan_pay_actual", rx_enc_impute_1p)
total_cost_actual = get_dist_cts("total_cost_actual", rx_enc_impute_1p)
plan_pay_clean = get_dist_cts("plan_pay_clean", rx_enc_impute_1p)
total_cost_clean = get_dist_cts("total_cost_clean", rx_enc_impute_1p)
cost_impute = get_dist_cts("cost_impute", rx_enc_impute_1p)
cost_impute_adj = get_dist_cts("cost_impute_adj", rx_enc_impute_1p)
cost_final = get_dist_cts("cost_final", rx_enc_impute_1p)
cost_ds_impute = get_dist_cts("cost_ds_impute", rx_enc_impute_1p)

median_unit_cost = get_dist_cts("median_per_unit", unit_price)
nadac_unit_cost = get_dist_cts("nadac_per_unit", unit_price)
asp_unit_cost = get_dist_cts("asp_per_unit", unit_price)
final_unit_cost = get_dist_cts("unit_price_final", unit_price)

sql = f"""
    {days_supply}
    union all
    {days_supply_pos}
    union all
    {days_supply_clean}
    union all
    {quantity_dispensed}
    union all
    {quantity_dispensed_pos}
    union all
    {quantity_final}
    union all
    {plan_pay_actual}
    union all
    {plan_pay_clean}
    union all
    {cost_impute}
    union all
    {cost_impute_adj}
    union all
    {cost_final}
    union all    
    {cost_ds_impute}
    union all
    {median_unit_cost}
    union all  
    {nadac_unit_cost}
    union all
    {asp_unit_cost}
    union all
    {final_unit_cost}
;
"""
df_dist = read_sql(sql)
display(df_dist)

,VAR,FILTER,N,N_NULL,N_NEG,N_ZERO,N_POS,MEAN,VARIANCE,MIN,P_01,P_05,P_10,P_25,MEDIAN,P_75,P_90,P_95,P_99,MAX
0,days_supply,TRUE,314533553,65320,7,22932,314445294,34.349787,6.725586e+02,-75.000000,1.000000,5.000000,7.000000,28.000000,30.000000,30.000000,90.00000,90.00000,90.00000,2.226000e+04
1,days_supply,days_supply > 0,314445294,0,0,0,314445294,34.352293,6.725215e+02,1.000000,1.000000,5.000000,7.000000,28.000000,30.000000,30.000000,90.00000,90.00000,90.00000,2.226000e+04
2,days_supply_clean,TRUE,319072399,106835,0,0,318965564,34.459951,6.546965e+02,1.000000,1.000000,5.000000,7.000000,28.000000,30.000000,30.000000,90.00000,90.00000,90.00000,3.650000e+02
3,quantity_dispensed,TRUE,314533553,787611,109328,3569085,310067529,499.061708,2.198883e+13,-337500.000000,0.000000,3.000000,9.000000,28.000000,30.000000,72.000000,100.00000,180.00000,360.00000,6.652101e+10
4,quantity_dispensed,quantity_dispensed > 0,310067529,0,0,0,310067529,505.023216,2.224969e+13,1.000000,1.000000,4.000000,10.000000,30.000000,30.000000,75.000000,100.00000,180.00000,360.00000,6.652101e+10
5,quantity_final,TRUE,319072399,1204109,0,0,317868290,59.114405,2.417623e+04,1.000000,1.000000,4.000000,10.000000,30.000000,30.000000,75.000000,93.00000,180.00000,354.00000,1.000000e+04
6,plan_pay_actual,TRUE,319072399,160006529,3756721,51010506,104298643,78.914683,6.527721e+06,-282322.000000,-6.000000,0.000000,0.000000,0.000000,4.000000,19.000000,113.00000,297.00000,1094.00000,1.011284e+07
7,plan_pay_clean,TRUE,319072399,167205552,0,51010506,100856341,66.121307,2.565507e+05,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,18.000000,109.00000,289.00000,981.00000,7.140000e+05
8,cost_impute,TRUE,319072399,1951543,0,0,317120856,92.448455,4.302177e+05,0.001057,1.000000,2.000000,2.333333,4.000000,9.000000,25.714286,153.00000,360.00000,1353.00000,7.189300e+05
9,cost_impute_adj,TRUE,319072399,1967645,0,0,317104754,92.450669,4.313772e+05,0.001057,1.000000,2.000000,2.333333,4.000000,9.000000,25.714286,153.00000,360.00000,1353.00000,7.189300e+05


CPU times: user 36.1 ms, sys: 562 µs, total: 36.7 ms
Wall time: 1min 1s


In [15]:
%%time
# final cost distribution by year
sql = f"""
    {get_dist_cts("cost_final", rx_enc_impute_1p, group_by_list = [f"year({claim_date})"])}
;
"""
df_dist = read_sql(sql)
display(df_dist)

,YEAR(DATE_OF_SERVICE),VAR,FILTER,N,N_NULL,N_NEG,N_ZERO,N_POS,MEAN,VARIANCE,MIN,P_01,P_05,P_10,P_25,MEDIAN,P_75,P_90,P_95,P_99,MAX
0,2000.0,cost_final,TRUE,1,0,0,1,0,0.000000,NaN,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000
1,2008.0,cost_final,TRUE,1,0,0,0,1,199.000000,NaN,199.0,199.0,199.0,199.0,199.000000,199.000000,199.0,199.0,199.000000,199.00,199.000000
2,2009.0,cost_final,TRUE,39,37,0,2,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000
3,2010.0,cost_final,TRUE,172,143,0,10,19,100.931034,218414.352217,0.0,0.0,0.0,0.0,0.000000,6.000000,15.0,52.4,84.200000,1847.88,2528.000000
4,2011.0,cost_final,TRUE,15941,10150,0,1945,3846,59.705578,170303.975995,0.0,0.0,0.0,0.0,0.000000,6.000000,30.0,134.0,240.500000,794.10,27076.000000
5,2012.0,cost_final,TRUE,7876844,438,0,2582423,5293983,51.793298,107131.525158,0.0,0.0,0.0,0.0,0.000000,5.000000,26.0,126.0,207.000000,609.00,131961.000000
6,2013.0,cost_final,TRUE,18300345,7846,0,3517665,14774834,57.359002,142605.118231,0.0,0.0,0.0,0.0,2.000000,7.500000,27.0,139.0,236.000000,687.00,580314.000000
7,2014.0,cost_final,TRUE,23871359,16331,0,3959638,19895390,65.030466,223304.132261,0.0,0.0,0.0,0.0,2.666667,7.700000,27.0,148.0,265.000000,792.00,297300.000000
8,2015.0,cost_final,TRUE,26196389,17187,0,4587558,21591644,72.487301,329452.059798,0.0,0.0,0.0,0.0,2.000000,7.000000,25.0,150.0,293.000000,930.00,457851.136613
9,2016.0,cost_final,TRUE,29968001,24182,0,4999346,24944473,75.797183,309111.330946,0.0,0.0,0.0,0.0,2.000000,6.000000,23.0,145.0,317.000000,1028.00,237553.000000


CPU times: user 37.1 ms, sys: 638 µs, total: 37.7 ms
Wall time: 41 s


In [16]:
%%time
# final cost distribution by year
sql = f"""
    {get_dist_cts("cost_impute", rx_enc_impute_1p, group_by_list = [f"year({claim_date})"])}
;
"""
df_dist = read_sql(sql)
display(df_dist)

,YEAR(DATE_OF_SERVICE),VAR,FILTER,N,N_NULL,N_NEG,N_ZERO,N_POS,MEAN,VARIANCE,MIN,P_01,P_05,P_10,P_25,MEDIAN,P_75,P_90,P_95,P_99,MAX
0,2000.0,cost_impute,TRUE,1,0,0,0,1,2.000000,NaN,2.000000,2.000000,2.00,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
1,2008.0,cost_impute,TRUE,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009.0,cost_impute,TRUE,39,0,0,0,39,138.763834,30705.698885,0.600000,0.828000,1.62,3.600000,5.500000,30.000000,240.900000,336.800000,559.946667,613.157333,624.000000
3,2010.0,cost_impute,TRUE,172,1,0,0,171,150.803254,378201.055663,0.642857,1.000000,2.00,3.000000,5.000000,12.000000,85.875000,231.000000,536.750000,2912.833333,6723.841667
4,2011.0,cost_impute,TRUE,15941,50,0,0,15891,86.568451,173883.887107,0.004333,1.333333,2.00,3.000000,5.333333,11.666667,50.806452,186.000000,318.954094,1064.300000,21003.107432
5,2012.0,cost_impute,TRUE,7876844,18932,0,0,7857912,61.025575,110954.354704,0.001057,1.200000,2.00,3.000000,4.200000,10.000000,35.250000,141.000000,225.000000,688.000000,88578.000000
6,2013.0,cost_impute,TRUE,18300345,58519,0,0,18241826,63.871728,125863.351338,0.003500,1.333333,2.00,3.000000,4.500000,10.000000,31.000000,145.000000,247.000000,733.583333,175707.000000
7,2014.0,cost_impute,TRUE,23871359,76837,0,0,23794522,72.005038,250877.112024,0.002333,1.333333,2.00,3.000000,4.200000,9.000000,30.000000,157.000000,279.000000,834.000000,297300.000000
8,2015.0,cost_impute,TRUE,26196389,73584,0,0,26122805,82.283193,383447.767436,0.002250,1.066667,2.00,2.222222,4.000000,8.333333,29.000000,179.000000,306.000000,1013.000000,446290.000000
9,2016.0,cost_impute,TRUE,29968001,72996,0,0,29895005,85.591323,360238.401447,0.002200,1.000000,2.00,2.000000,4.000000,8.000000,27.000000,174.000000,327.000000,1160.640000,279270.000000


CPU times: user 33.4 ms, sys: 471 µs, total: 33.8 ms
Wall time: 40.8 s


In [17]:
# check frequency of imputation performed
sql = f"""
    select cost_final_source, count(*) as n_claim, count(distinct {ndc_var}) as n_ndc
    from {rx_enc_impute}_1p
    group by cost_final_source
    order by cost_final_source
    ;
"""

df = read_sql(sql)
display(df)

sql = f"""
    select unit_price_source, count(*) as n_claim, count(distinct {ndc_var}) as n_ndc
    from {rx_enc_impute}_1p
    group by unit_price_source
    order by unit_price_source
    ;
"""

df = read_sql(sql)
display(df)

sql = f"""
    select quantity_source, count(*) as n_claim, count(distinct {ndc_var}) as n_ndc
    from {rx_enc_impute}_1p
    group by quantity_source
    order by quantity_source
    ;
"""

df = read_sql(sql)
display(df)

sql = f"""
    select cost_final_source, unit_price_source, quantity_source, count(*) as n_claim, count(distinct {ndc_var}) as n_ndc
    from {rx_enc_impute}_1p
    group by cost_final_source, unit_price_source, quantity_source
    order by cost_final_source, unit_price_source, quantity_source
    ;
"""

df = read_sql(sql)
display(df)

,COST_FINAL_SOURCE,N_CLAIM,N_NDC
0,imp,166219975,92205
1,orig,151727219,115962
2,None,1125205,17091


,UNIT_PRICE_SOURCE,N_CLAIM,N_NDC
0,asp,39374,329
1,median,408254,50812
2,median_by_year,316426681,98825
3,nadac,253039,106
4,None,1945051,23106


,QUANTITY_SOURCE,N_CLAIM,N_NDC
0,drop,1199607,820
1,imp,125975,21465
2,imp_ds,11012268,59123
3,missing,4502,224
4,orig,306730047,137814


,COST_FINAL_SOURCE,UNIT_PRICE_SOURCE,QUANTITY_SOURCE,N_CLAIM,N_NDC
0,imp,asp,imp,15,6
1,imp,asp,imp_ds,1615,121
2,imp,asp,orig,32516,286
3,imp,median,imp,3864,1355
4,imp,median,imp_ds,21718,6225
5,imp,median,orig,250532,28516
6,imp,median_by_year,imp,89129,13533
7,imp,median_by_year,imp_ds,6992779,46667
8,imp,median_by_year,orig,158610935,77153
9,imp,nadac,imp,48,28


In [18]:
%%time
# final cost distribution by year
sql = f"""
    select cost_final_source, count(*) as n_claim, count(distinct {ndc_var}) as n_ndc
    from {rx_enc_impute_1p}
    where cost_final = 0
    group by cost_final_source
    order by cost_final_source
;
"""
df_dist = read_sql(sql)
display(df_dist)

,COST_FINAL_SOURCE,N_CLAIM,N_NDC
0,orig,51010506,87737


CPU times: user 11.8 ms, sys: 0 ns, total: 11.8 ms
Wall time: 4.81 s


In [24]:
%%time
# compare imputed vs cleaned actual
sql = f"""
    with cost_diff as (
    select cost_impute - plan_pay_clean as diff, quantity_source, unit_price_source
    from {rx_enc_impute}_1p
    where cost_impute > 0 and plan_pay_clean > 0
    )
    {get_dist_cts("diff", 'cost_diff', group_by_list = ['quantity_source', 'unit_price_source'])}
    ;
"""

df = read_sql(sql)
display(df)

sql = f"""
    with cost_diff as (
    select (cost_impute - plan_pay_clean)/plan_pay_clean as relative_diff, quantity_source, unit_price_source
    from {rx_enc_impute}_1p
    where cost_impute > 0 and plan_pay_clean > 0
    )
    {get_dist_cts("relative_diff", 'cost_diff', group_by_list = ['quantity_source', 'unit_price_source'])}
    ;
"""

df = read_sql(sql)
display(df)

,QUANTITY_SOURCE,UNIT_PRICE_SOURCE,VAR,FILTER,N,N_NULL,N_NEG,N_ZERO,N_POS,MEAN,VARIANCE,MIN,P_01,P_05,P_10,P_25,MEDIAN,P_75,P_90,P_95,P_99,MAX
0,imp,median,diff,TRUE,375,0,187,11,177,-2073.187583,4.762207e+08,-360503.000000,-51184.660000,-844.84500,-148.400000,-16.3,0.000000,9.750000,67.628571,263.000,9777.209335,32465.0
1,imp,median_by_year,diff,TRUE,12082,0,4978,516,6588,12.563442,3.129243e+07,-475347.000000,-772.660000,-117.66294,-40.000000,-6.0,1.285714,16.000000,128.000000,332.550,5211.590000,70122.0
2,imp_ds,median,diff,TRUE,1586,0,813,35,738,-458.239709,8.290354e+07,-109034.666667,-36433.909062,-1883.83046,-282.333333,-31.0,-0.612500,13.000000,120.000000,610.625,17681.650000,145605.0
3,imp_ds,median_by_year,diff,TRUE,2055418,0,969342,117387,968689,16.258967,6.857503e+05,-404607.000000,-500.000000,-54.00000,-16.000000,-4.0,0.000000,4.333333,34.000000,146.000,563.000000,110127.5
4,orig,median,diff,TRUE,38064,0,17721,1937,18406,51.866797,4.282374e+06,-58646.000000,-898.964643,-95.00000,-29.235000,-4.0,0.000000,5.000000,38.500000,122.940,1363.774000,279135.0
5,orig,median_by_year,diff,TRUE,98191235,0,45229333,8004562,44957340,12.460623,8.690274e+04,-455826.000000,-92.000000,-29.00000,-14.000000,-4.0,0.000000,4.000000,16.000000,40.000,273.000000,143076.0


,QUANTITY_SOURCE,UNIT_PRICE_SOURCE,VAR,FILTER,N,N_NULL,N_NEG,N_ZERO,N_POS,MEAN,VARIANCE,MIN,P_01,P_05,P_10,P_25,MEDIAN,P_75,P_90,P_95,P_99,MAX
0,imp,median,relative_diff,TRUE,375,0,187,11,177,1.281453,29.602765,-0.999612,-0.992154,-0.963353,-0.918095,-0.606858,0.000000,1.371711,3.712381,7.244118,18.819333,78.366927
1,imp,median_by_year,relative_diff,TRUE,12082,0,4978,516,6588,2.858777,702.120056,-0.999940,-0.987500,-0.880488,-0.750000,-0.500000,0.321140,2.355659,6.000000,10.000000,28.325000,1291.568182
2,imp_ds,median,relative_diff,TRUE,1586,0,813,35,738,5.651862,4927.862076,-0.999441,-0.970012,-0.884086,-0.777778,-0.498296,-0.034934,1.000000,3.859951,8.842563,79.244326,2335.533333
3,imp_ds,median_by_year,relative_diff,TRUE,2055418,0,969342,117387,968689,1.430663,785.727328,-0.999780,-0.872275,-0.750000,-0.666667,-0.440000,0.000000,0.987500,3.044444,5.666667,21.000000,19997.000000
4,orig,median,relative_diff,TRUE,38064,0,17721,1937,18406,2.104107,1333.701082,-0.999924,-0.890277,-0.646091,-0.500000,-0.232100,0.000000,0.442671,1.666667,3.714286,24.778427,2996.000000
5,orig,median_by_year,relative_diff,TRUE,98191235,0,45229333,8004562,44957340,0.860256,226.661552,-0.999892,-0.800000,-0.619048,-0.500000,-0.230769,0.000000,0.500000,2.000000,3.500000,12.333333,24296.000000


CPU times: user 277 ms, sys: 0 ns, total: 277 ms
Wall time: 559 ms


In [25]:
%%time
# compare imputed vs cleaned actual
sql = f"""
    with cost_diff as (
    select cost_ds_impute - plan_pay_clean as diff
    from {rx_enc_impute}_1p
    where cost_ds_impute > 0 and plan_pay_clean > 0
    )
    {get_dist_cts("diff", 'cost_diff')}
    ;
"""

df = read_sql(sql)
display(df)


,VAR,FILTER,N,N_NULL,N_NEG,N_ZERO,N_POS,MEAN,VARIANCE,MIN,P_01,P_05,P_10,P_25,MEDIAN,P_75,P_90,P_95,P_99,MAX
0,diff,TRUE,100621953,0,45553301,10023655,45044997,2.417909,107684.606427,-459180.0,-283.0,-61.0,-24.0,-6.0,0.0,4.0,18.0,53.0,301.0,142420.5


CPU times: user 21.4 ms, sys: 0 ns, total: 21.4 ms
Wall time: 235 ms


In [26]:
# check price year and inflation adjustment
sql = f"""
    select price_year, count(*) as n_claim, count(distinct {ndc_var}) as n_ndc
    from {rx_enc_impute}_1p
    group by price_year
    order by price_year
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select price_year, count(*)
    from {unit_price}
    group by price_year
    order by price_year
    ;
"""

df = read_sql(sql)
display(df)

,PRICE_YEAR,N_CLAIM,N_NDC
0,2012.0,7872205,30499
1,2013.0,18250494,42462
2,2014.0,23785909,46705
3,2015.0,26118391,48738
4,2016.0,29994500,50344
5,2017.0,37845576,51448
6,2018.0,36888019,52568
7,2019.0,37667663,54424
8,2020.0,39246382,55700
9,2021.0,42148197,55279


,PRICE_YEAR,COUNT(*)
0,2012,221346
1,2013,149947
2,2014,138169
3,2015,135555
4,2016,133381
5,2017,133955
6,2018,134641
7,2019,136503
8,2020,138879
9,2021,157101


In [27]:
# find a list of NDC with crazy large price or quite different from original
sql = f"""
    select o.{ndc_var}, dm.ndc, dm.active, dm.cui_l1_name, dm.brand_generic, n, avg_quantity, avg_plan_pay, avg_impute_adj, avg_cost_final
    from (
        select {ndc_var}, count(*) as n, 
            avg(quantity_final) as avg_quantity, 
            avg(plan_pay_clean) as avg_plan_pay, 
            avg(cost_impute_adj) as avg_impute_adj, 
            avg(cost_final) as avg_cost_final
        from {rx_enc_impute}
        where cost_final >= 100000 or cost_final_sentinel >= 100000
        group by {ndc_var}
    ) o
    left join {dm_file} dm
    on o.{ndc_var} = dm.ndc
    having dm.ndc is not null
    order by avg_cost_final desc
    ;
"""

df = read_sql(sql)
display(df)

,NDC11,NDC,ACTIVE,CUI_L1_NAME,BRAND_GENERIC,N,AVG_QUANTITY,AVG_PLAN_PAY,AVG_IMPUTE_ADJ,AVG_COST_FINAL
0,71894013206,71894013206,True,{6 (8.3 ml onasemnogene abeparvovec-xioi 20000...,brand,3,9.000000,850000.000000,850000.000000,850000.000000
1,71894012604,71894012604,True,{4 (8.3 ml onasemnogene abeparvovec-xioi 20000...,brand,6,1.000000,731000.000000,733501.737880,737156.737880
2,71894012002,71894012002,True,{2 (8.3 ml onasemnogene abeparvovec-xioi 20000...,brand,2,9.000000,713970.000000,713970.000000,713970.000000
3,71894012404,71894012404,True,{2 (5.5 ml onasemnogene abeparvovec-xioi 20000...,brand,2,9.000000,713970.000000,713970.000000,713970.000000
4,71894012303,71894012303,True,{3 (8.3 ml onasemnogene abeparvovec-xioi 20000...,brand,9,1.000000,714000.000000,687735.449735,687735.449735
5,00173089601,00173089601,True,belantamab mafodotin-blmf 100 mg injection [bl...,brand,11,60.000000,NaN,532320.000000,532320.000000
6,00078084619,00078084619,True,tisagenlecleucel 250000000 cells injection [ky...,brand,40,1.025000,473116.000000,495043.911087,495043.911087
7,69468015120,69468015120,True,uridine triacetate 10000 mg oral granules [vis...,brand,3,120.000000,NaN,479833.607231,479833.607231
8,71894013307,71894013307,True,{2 (5.5 ml onasemnogene abeparvovec-xioi 20000...,brand,9,9.000000,357000.000000,714000.000000,476000.000000
9,00187001110,00187001110,False,desirudin 15 mg injection [iprivask],brand,46,2017.826087,418058.000000,452654.442029,434589.572464


In [28]:
# find a list of NDC with crazy large price or quite different from original
sql = f"""
    select *
    from {rx_enc_impute}_1p
    where (cost_final > 100000 or cost_final_sentinel > 100000)
    order by cost_final desc
    ;
"""

# RX_ENCOUNTERS_20220511_IMPUTED_1P

df_ndc9 = read_sql(sql)
display(df_ndc9)

,UPK_KEY2,CLAIM_ID,DATE_OF_SERVICE,NDC11,NDC9,PLAN_PAY_ACTUAL,PLAN_PAY_CLEAN,TOTAL_COST_ACTUAL,TOTAL_COST_CLEAN,COST_IMPUTE,PRICE_YEAR,COST_IMPUTE_ADJ,COST_FINAL,COST_FINAL_SENTINEL,COST_FINAL_ROUND,COST_FINAL_SENTINEL_ROUND,COST_FINAL_SOURCE,COST_DS_IMPUTE,PRICE_YEAR_CPI_RATIO,CLAIM_YEAR_CPI_RATIO,QUANTITY_FINAL,QUANTITY_SOURCE,UNIT_PRICE_FINAL,UNIT_PRICE_SOURCE,MEDIAN_PER_UNIT,NADAC_PER_UNIT,ASP_PER_UNIT,UNIT_OF_MEASURE,PAYER_TYPE_CODE,PATIENT_STATE,SOURCE,DFG_NAME_ARRAY
0,xLjibnmql8zh38kWoIPfPzgy6Ccs3mc3x6CiEafvg0c=,e70bb48c5bcd8a85036c4f47f113479b902e78df,2021-06-18,71894013307,718940133,714000.0,714000.0,714000.0,714000.0,714000.0,2021.0,714000.000000,714000.000000,714000.000000,714000.00,714000.00,orig,714000.0,1.0000,1.0000,9.0,orig,79333.333333,median_by_year,79333.333333,None,NaN,EA,None,AZ,fleming-nightingale,None
1,WWSdYZOxQYhBPOjO13U64sH36WJejj+mztbgAZcb5MQ=,11b7800a825ffa1e5e6829bcaa73b7f82b43a3ea,2018-12-21,00169720501,001697205,621936.0,621936.0,621936.0,621936.0,146589.0,2018.0,146589.000000,621936.000000,146589.000000,621936.00,146589.00,orig,621936.0,1.0906,1.0906,10000.0,imp,14.658900,median_by_year,14.658900,None,2.307,None,None,CA,fleming-nightingale,"[\n ""Injectable Product""\n]"
2,WWSdYZOxQYhBPOjO13U64sH36WJejj+mztbgAZcb5MQ=,5a3fe190c7a008716d85bd3a23615cc08abad6a6,2013-10-31,00169720501,001697205,580314.0,580314.0,580314.0,580314.0,175707.0,2013.0,175707.000000,580314.000000,175707.000000,580314.00,175707.00,orig,194980.0,1.2440,1.2440,10000.0,imp_ds,17.570700,median_by_year,17.570700,None,2.307,None,None,CA,fleming-nightingale,"[\n ""Injectable Product""\n]"
3,trezJTJ/gg3qECH0hAvpsP4zPvOfw2wxK0K7Wbe3qi8=,a3e406f23f4939bf3d9adb5d7c7f9338c90c62ec,2021-12-13,72171050501,721710505,NaN,NaN,NaN,NaN,445676.4,2019.0,471703.901760,471703.901760,471703.901760,471703.90,471703.90,imp,NaN,1.0584,1.0000,120.0,orig,3713.970000,median_by_year,3713.970000,None,362.313,None,M,AZ,rowley,"[\n ""Injectable Product""\n]"
4,Nvh+Bbs0iFDOFxY+GVYgTNEHE5Qy+agFjcWPNWDOV/0=,558d3ea6e55da0835445d2c223d7e108728dfe94,2015-05-27,50242005656,502420056,NaN,NaN,NaN,NaN,446290.0,2014.0,457851.136613,457851.136613,457851.136613,457851.14,457851.14,imp,6754.0,1.2158,1.1851,130.0,orig,3433.000000,median,3433.000000,None,NaN,None,M,GA,rowley,"[\n ""Injectable Product""\n]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,v96lcQDO3gb7u1EUVA/AqtMLiIPPAmuya+GBSKOYMKI=,a814ea440a718b506ad7696336f9367317020c99,2018-05-07,58597830201,585978302,0.0,0.0,70.0,70.0,183260.0,2021.0,168035.943517,0.000000,168035.943517,0.00,168035.94,orig,NaN,1.0000,1.0906,5.0,orig,36652.000000,median_by_year,36652.000000,None,NaN,ML,None,None,fleming-nightingale,None
462,4rzk9jPanEWdfIYBv+8KLvW3Z7pIOiKdIJXhEh569lw=,b729a7a5dc1b33fb499d319cfcfef63856131573,2019-06-13,64406005801,644060058,0.0,0.0,0.0,0.0,125016.0,2019.0,125016.000000,0.000000,125016.000000,0.00,125016.00,orig,128521.0,1.0584,1.0584,5.0,orig,25003.200000,median_by_year,25003.200000,None,NaN,None,M,FL,rowley,"[\n ""Injectable Product""\n]"
463,DM961Jl7UlxhVFNY4HV1E6LuRfzC1DaG2CwgBNQdFJk=,3a53cbd25a39cd410fc55b09caf09772892c8cef,2020-09-24,75987010108,759870101,0.0,0.0,0.0,0.0,118047.0,2020.0,118047.000000,0.000000,118047.000000,0.00,118047.00,orig,52466.0,1.0116,1.0116,1080.0,orig,109.302778,median_by_year,109.302778,None,NaN,None,M,MS,rowley,"[\n ""Oral Product"",\n ""Pill""\n]"
464,UiYUGdJ4LML5bUpZIzDXjFY5tsHBkKfTp1cKj+ONIz8=,42a90499b4c57bdde262659841af6b44c506ab1f,2022-04-15,62991117903,629911179,0.0,0.0,116.0,116.0,454520.0,2019.0,494108.430567,0.000000,494108.430567,0.00,494108.43,orig,NaN,1.0584,0.9736,600.0,orig,757.533333,median,757.533333,None,NaN,None,None,NJ,fleming-nightingale,None


In [243]:
%%time
# a working template to study one particular type of drug to resolve outliers
sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where encounter_key = 'da4c2b065b1961320fce0fe61d6e6c3c'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {unit_price}
    where ndc = '54868075300'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where ndc = '54868075300' and plan_pay_actual > 0
    ;
"""
df = read_sql(sql)
display(df)

quantity_dispensed_ndc = get_dist_cts("quantity_dispensed", rx_enc_step1, filter = "ndc = '54868075300'", filter_print = 'ndc')
sql = f"""
    {quantity_dispensed_ndc}
    ;
"""
df = read_sql(sql)
display(df)

quantity_dispensed_ndc = get_dist_cts("PLAN_PAY_CLEAN", rx_enc_step1, filter = "ndc = '54868075300'", filter_print = 'ndc')
sql = f"""
    {quantity_dispensed_ndc}
    ;
"""
df = read_sql(sql)
display(df)

,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PAYER_TYPE_CODE,PATIENT_STATE,SOURCES,PHARMACY_SUBMITTED_COST,COPAY_COINSURANCE,PLAN_PAY_PRIMARY,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_10,QUANTITY_50,QUANTITY_90,QUANTITY_CLEAN,PLAN_PAY_10,PLAN_PAY_50,PLAN_PAY_90,PLAN_PAY_CLEAN,TOTAL_COST_10,TOTAL_COST_50,TOTAL_COST_90,TOTAL_COST_CLEAN
0,548680753,9QkqTapCVZsGxERvSe3W+XGIx+vo3XL6VydC42U6CN4=,da4c2b065b1961320fce0fe61d6e6c3c,2014-06-03,54868075300,548680753,1,1200000.0,None,M,GA,"[\n ""rowley""\n]",None,None,None,None,None,None,1,2.0,1200000.0,1200000.0,1200000.0,13.4,47.0,387.0,None,33.4,67.0,768.6,None


,NDC9,NDC,N_CLAIM,START_DATE,END_DATE,UNIT_PRICE_FINAL,UNIT_PRICE_SOURCE,PRICE_YEAR,MEDIAN_PER_UNIT,NADAC_PER_UNIT,ASP_PER_UNIT,MEDIAN_PLAN_PAY_PER_UNIT,MEDIAN_PLAN_PAY_PER_UNIT_BY_YEAR
0,548680753,54868075300,9,1970-01-01,2012-12-31,21.5,median,2013,21.5,None,None,21.5,24.75
1,548680753,54868075300,9,2016-01-01,2017-12-31,21.5,median,2016,21.5,None,None,21.5,18.50
2,548680753,54868075300,9,2020-01-01,2030-12-31,21.5,median,2019,21.5,None,None,21.5,20.50
3,548680753,54868075300,9,2013-01-01,2013-12-31,21.5,median,2013,21.5,None,None,21.5,24.75
4,548680753,54868075300,9,2014-01-01,2015-12-31,21.5,median_by_year,2014,21.5,None,None,21.5,21.50
5,548680753,54868075300,9,2018-01-01,2019-12-31,21.5,median,2018,21.5,None,None,21.5,20.50


,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PAYER_TYPE_CODE,PATIENT_STATE,SOURCES,PHARMACY_SUBMITTED_COST,COPAY_COINSURANCE,PLAN_PAY_PRIMARY,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_10,QUANTITY_50,QUANTITY_90,QUANTITY_CLEAN,PLAN_PAY_10,PLAN_PAY_50,PLAN_PAY_90,PLAN_PAY_CLEAN,TOTAL_COST_10,TOTAL_COST_50,TOTAL_COST_90,TOTAL_COST_CLEAN


,VAR,FILTER,N,N_NULL,N_NEG,N_ZERO,N_POS,MEAN,VARIANCE,MIN,P_01,P_05,P_10,P_25,MEDIAN,P_75,P_90,P_95,P_99,MAX
0,quantity_dispensed,ndc,97,0,0,0,97,884639.917526,3.680113e+11,1.0,1.0,1.0,2.0,4.0,1200000.0,1200000.0,1200000.0,1200000.0,2400000.0,2400000.0


,VAR,FILTER,N,N_NULL,N_NEG,N_ZERO,N_POS,MEAN,VARIANCE,MIN,P_01,P_05,P_10,P_25,MEDIAN,P_75,P_90,P_95,P_99,MAX
0,PLAN_PAY_CLEAN,ndc,97,96,0,0,1,47.0,None,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0


CPU times: user 326 ms, sys: 4.18 ms, total: 330 ms
Wall time: 1min 11s


In [ ]:
# MISC investigations below

In [39]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select ndc9, count(distinct ndc) as n_ndc, VARIANCE(nadac_per_unit) as var
    from 
    (
    select left(ndc, 9) as ndc9, ndc, avg(nadac_per_unit) as nadac_per_unit
    from {nadac_per_unit_price}
    group by ndc9, ndc
    )
    group by ndc9
    having n_ndc > 1
    order by var desc
    limit 10
    ;
"""
df_ndc9 = read_sql(sql)
display(df_ndc9)

,NDC9,N_NDC,VAR
0,000851341,3,4451.410105
1,782060114,3,4451.410105
2,502420860,2,2365.455625
3,669930084,2,1026.400183
4,517590202,2,96.311331
5,005740820,2,75.768265
6,525360625,2,75.768265
7,690970802,2,75.768265
8,005740827,2,75.768265
9,674570409,2,65.270340


CPU times: user 12.7 ms, sys: 0 ns, total: 12.7 ms
Wall time: 1.37 s


In [42]:
sql = f"""
    select *
    from {nadac_per_unit_price}
    where left(ndc, 9) = '000851341'
    ;
"""
read_sql(sql)

,NDC,NADAC_PER_UNIT,NDC_DESCRIPTION,OTC,PRICING_UNIT,CLASSIFICATION_FOR_RATE_SETTING,START_DATE,END_DATE
0,00085134101,309.71889,ASMANEX TWISTHALR 220 MCG #120,N,EA,B,1970-01-01,2021-10-19
1,00085134101,309.71889,ASMANEX TWISTHALR 220 MCG #120,N,EA,B,2021-10-20,2022-01-13
2,00085134101,324.89512,ASMANEX TWISTHALR 220 MCG #120,N,EA,B,2022-01-14,2030-12-31
3,00085134102,215.36696,ASMANEX TWISTHALER 220 MCG #60,N,EA,B,1970-01-01,2021-12-21
4,00085134102,215.36696,ASMANEX TWISTHALER 220 MCG #60,N,EA,B,2021-12-22,2022-01-13
5,00085134102,225.91994,ASMANEX TWISTHALER 220 MCG #60,N,EA,B,2022-01-14,2030-12-31
6,00085134107,183.475,ASMANEX TWISTHALER 220 MCG #30,N,EA,B,1970-01-01,2021-12-21
7,00085134107,183.475,ASMANEX TWISTHALER 220 MCG #30,N,EA,B,2021-12-22,2022-01-13
8,00085134107,192.46528,ASMANEX TWISTHALER 220 MCG #30,N,EA,B,2022-01-14,2030-12-31


In [43]:
sql = f"""
    select *
    from {rx_enc_step1_1p}
    where ndc9 = '000851341'
    ;
"""
read_sql(sql)

,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PHARMACY_SUBMITTED_COST,COPAY_COINSURANCE,...,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_99,QUANTITY_CLEAN,PLAN_PAY_99,PLAN_PAY_CLEAN,TOTAL_COST_99,TOTAL_COST_CLEAN
0,gld39m0+mvcoc2gL1gny5pqPL+BcTFuTEmC9FvnyjGk=,9442283d58950263b5d34359c4a93f5e,2016-02-16,00085134107,000851341,30,1.0,None,NaN,0.0,...,NaN,166.0,166.0,30.0,3.0,1.0,513.0,166.0,549.0,166.0
1,Rdwhf/bkjzfkEJscGLBDkJ0K0dSD55LZeUNWmweDvTw=,428df8533f907ff57b74c7e20491a696,2019-07-10,00085134101,000851341,30,1.0,None,298.0,0.0,...,107.0,191.0,298.0,30.0,3.0,1.0,868.0,191.0,910.0,298.0
2,qZAW5tJCfd4rFSreZh0KbzS16qRYb3IhnRvDS2TZEFI=,70503acb87cef3bd67d201ea82877d6b,2019-05-09,00085134107,000851341,30,1.0,EA,231.0,45.0,...,45.0,143.0,188.0,30.0,3.0,1.0,513.0,143.0,549.0,188.0
3,qKMnCgvfOnoFuzTvgNhKMmBMyhE3rQ3Oe7cQxAoLq/E=,7f1e040a695fbd1aaac5925c612d26b8,2019-12-16,00085134102,000851341,90,3.0,EA,812.0,120.0,...,120.0,519.0,639.0,90.0,3.0,3.0,590.0,519.0,634.0,NaN
4,YtXBe/+/5cDMGN0ChcQVGJFwDs7HUzdYC0LbcbrWZH0=,1d592a694f8625b304a85dce5035513e,2020-03-17,00085134107,000851341,30,1.0,EA,231.0,NaN,...,0.0,188.0,188.0,30.0,3.0,1.0,513.0,188.0,549.0,188.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29277,3Ppu7N3JK7FZIlwYWqLThRYnAYdjh94tS1P5OrGA1/Q=,b974cb44c4600cac0e7115002c22731e,2016-07-05,00085134102,000851341,60,2.0,EA,484.0,10.0,...,10.0,394.0,404.0,60.0,3.0,2.0,590.0,394.0,634.0,404.0
29278,F//ajRBX4va4qQTgVGOqbzVTdptHmNALBKCIykAunik=,5939558705313331681596327f33d7b3,2016-12-28,00085134101,000851341,30,1.0,EA,366.0,NaN,...,0.0,322.0,322.0,30.0,3.0,1.0,868.0,322.0,910.0,322.0
29279,AmpmAXsw+RdM5pV3OHLcugyH3fxHvd/qxnzDaPCQVc8=,71a9fc0a9dc6f62bbfd402eeffc5b0c1,2017-01-25,00085134102,000851341,30,1.0,EA,252.0,30.0,...,30.0,178.0,208.0,30.0,3.0,1.0,590.0,178.0,634.0,208.0
29280,CrOxQsT4UeojCJ86hyWeQwbkkj21/VLL2vurufpQgik=,513b503b7970565d12ae9cd0712d8cb0,2021-01-28,00085134107,000851341,30,1.0,EA,NaN,30.0,...,NaN,NaN,NaN,30.0,3.0,1.0,513.0,NaN,549.0,NaN


In [54]:
sql = f"""
    select *
    from {nadac_per_unit_price}
    where left(ndc, 9) = '000744339'
    ;
"""
read_sql(sql)

,NDC,NADAC_PER_UNIT,NDC_DESCRIPTION,OTC,PRICING_UNIT,CLASSIFICATION_FOR_RATE_SETTING,START_DATE,END_DATE
0,00074433902,2904.5386399999998,HUMIRA PEN 40 MG/0.8 ML,N,EA,B,1970-01-01,2021-10-19
1,00074433902,2904.5386399999998,HUMIRA PEN 40 MG/0.8 ML,N,EA,B,2021-10-20,2022-01-13
2,00074433902,3119.4745,HUMIRA PEN 40 MG/0.8 ML,N,EA,B,2022-01-14,2030-12-31
3,00074433906,2900.60425,HUMIRA PEN CROHN'S-UC-HS STARTER 40 MG/0.8 ML,N,EA,B,1970-01-01,2020-12-31
4,00074433906,2900.60425,HUMIRA PEN CROHN'S-UC-HS STARTER 40 MG/0.8 ML,N,EA,B,2021-01-01,2022-01-13
5,00074433906,3115.2489600000004,HUMIRA PEN CROHN'S-UC-HS STARTER 40 MG/0.8 ML,N,EA,B,2022-01-14,2030-12-31
6,00074433907,2900.65499,HUMIRA PEN PSORIASIS-UVEITIS-ADOL HS STARTER 4...,N,EA,B,1970-01-01,2020-12-31
7,00074433907,2900.65499,HUMIRA PEN PSORIASIS-UVEITIS-ADOL HS STARTER 4...,N,EA,B,2021-01-01,2022-01-13
8,00074433907,3115.30346,HUMIRA PEN PSORIASIS-UVEITIS-ADOL HS STARTER 4...,N,EA,B,2022-01-14,2030-12-31


In [86]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select ndc9, PRICING_UNIT, count(distinct ndc) as n_ndc, VARIANCE(nadac_per_unit) as var
    from 
    (
    select left(ndc, 9) as ndc9, ndc, PRICING_UNIT, avg(nadac_per_unit) as nadac_per_unit
    from {nadac_per_unit_price}
    group by ndc9, ndc, PRICING_UNIT
    )
    group by ndc9, PRICING_UNIT
    order by var desc
    ;
"""
df_ndc9 = read_sql(sql)
display(df_ndc9)

,NDC9,PRICING_UNIT,N_NDC,VAR
0,637390969,EA,1,NaN
1,005170440,ML,1,NaN
2,707101230,EA,1,NaN
3,502420828,EA,1,NaN
4,707101609,EA,1,NaN
...,...,...,...,...
16538,683820010,EA,2,0.0
16539,633040090,EA,2,0.0
16540,623320056,EA,2,0.0
16541,586570703,EA,2,0.0


CPU times: user 267 ms, sys: 0 ns, total: 267 ms
Wall time: 888 ms


In [81]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select left(ndc, 9) as ndc9, ndc, avg(nadac_per_unit) as nadac_per_unit
    from {nadac_per_unit_price}
    where ndc9 = '167140294'
    group by ndc9, ndc
    ;
"""
df_ndc9 = read_sql(sql)
display(df_ndc9)

,NDC9,NDC,NADAC_PER_UNIT
0,167140294,16714029401,0.082952
1,167140294,16714029402,0.067085
2,167140294,16714029403,0.062205


CPU times: user 11.7 ms, sys: 196 µs, total: 11.9 ms
Wall time: 155 ms


In [84]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select left(ndc, 9) as ndc9, *
    from {nadac_per_unit_price}
    where ndc9 = '762820628'
    ;
"""
df_ndc9 = read_sql(sql)
display(df_ndc9)

,NDC9,NDC,NADAC_PER_UNIT,NDC_DESCRIPTION,OTC,PRICING_UNIT,CLASSIFICATION_FOR_RATE_SETTING,START_DATE,END_DATE
0,762820628,76282062810,0.0246,CITALOPRAM HBR 10 MG TABLET,N,EA,G,1970-01-01,2021-12-21
1,762820628,76282062810,0.0246,CITALOPRAM HBR 10 MG TABLET,N,EA,G,2021-12-22,2022-01-18
2,762820628,76282062810,0.026,CITALOPRAM HBR 10 MG TABLET,N,EA,G,2022-01-19,2022-02-22
3,762820628,76282062810,0.026369999999999998,CITALOPRAM HBR 10 MG TABLET,N,EA,G,2022-02-23,2030-12-31
4,762820628,76282062890,0.026,CITALOPRAM HBR 10 MG TABLET,N,EA,G,1970-01-01,2022-02-15
5,762820628,76282062890,0.026,CITALOPRAM HBR 10 MG TABLET,N,EA,G,2022-02-16,2022-02-22
6,762820628,76282062890,0.026369999999999998,CITALOPRAM HBR 10 MG TABLET,N,EA,G,2022-02-23,2030-12-31


CPU times: user 16.1 ms, sys: 0 ns, total: 16.1 ms
Wall time: 174 ms


In [44]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select pricing_unit, count(*)
    from {nadac_per_unit_price}
    group by pricing_unit
    ;
"""
df_ndc9 = read_sql(sql)
display(df_ndc9)

,PRICING_UNIT,COUNT(*)
0,ML,9079
1,EA,79524
2,GM,5993


CPU times: user 236 ms, sys: 0 ns, total: 236 ms
Wall time: 15.7 s


In [103]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select left(ndc, 9) as ndc9, *
    from {unit_price}
    where ndc = '00003021501'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where ndc = '00003021501'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where encounter_key = '2128c458187afdf637749d3eb97f2a17'
    ;
"""
df = read_sql(sql)
display(df)

,NDC9,NDC,N_CLAIM,START_DATE,END_DATE,UNIT_PRICE_FINAL,MEDIAN_TOTAL_COST_PER_UNIT,MEDIAN_PLAN_PAY_PER_UNIT,NADAC_PER_UNIT,ASP_PER_UNIT
0,000030215,00003021501,35923,1970-01-01,2030-12-31,748.785714,748.785714,748.785714,None,None


,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PHARMACY_SUBMITTED_COST,...,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_99,QUANTITY_CLEAN,PLAN_PAY_99,PLAN_PAY_CLEAN,TOTAL_COST_99,TOTAL_COST_CLEAN
0,000030215,8V0F9MrqR321YIHuXg6Tcf8/ROdP0ZLatnSkBGrXxGA=,30a9925022a3b6931fa4fac7710f7b64,2016-06-03,00003021501,000030215,28,28.0,None,NaN,...,NaN,21105.0,21105.0,28,282.0,28.0,22228.0,21105.0,22315.0,21105.0
1,000030215,zmlpJdflbvQeR2ZNzArGOYF7oGRukBNgr9eSsUu7UWw=,df6ed94f146488e5f482040433e79f3c,2015-09-21,00003021501,000030215,28,28.0,None,24144.0,...,25.0,21181.0,21206.0,28,282.0,28.0,22228.0,21181.0,22315.0,21206.0
2,000030215,pzmOiQxXkM9EUccHlmKs+3Y3qoNZuUNc4LfCFY6KCDw=,18b0baf56033adc4db9e128e27f8a945,2015-11-09,00003021501,000030215,28,28.0,EA,25200.0,...,0.0,22090.0,22090.0,28,282.0,28.0,22228.0,22090.0,22315.0,22090.0
3,000030215,1UH29CqVLZtCERUrA3JsGEMuE9pxBqIQo0iK4ccHinE=,20df6fbb54395b57d4809a5662aaa2fe,2015-12-15,00003021501,000030215,28,28.0,None,25210.0,...,0.0,22180.0,22180.0,28,282.0,28.0,22228.0,22180.0,22315.0,22180.0
4,000030215,tR/JHyQOjdYSN2zHEkto4cyuUyD0GRaytbF3CwEUgKQ=,d9c9360e7f7b1ec67a1a58a17fdfd814,2016-01-08,00003021501,000030215,14,14.0,None,NaN,...,NaN,10238.0,10238.0,14,282.0,14.0,22228.0,10238.0,22315.0,10238.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,000030215,zQjWQPk/5K+EFeyd/GMKu3NIam1siArnLowfY4OvDVQ=,34002a6113765fa98fda06897ec41fee,2015-10-23,00003021501,000030215,28,28.0,None,NaN,...,NaN,NaN,NaN,28,282.0,28.0,22228.0,NaN,22315.0,NaN
382,000030215,HOqNq/UNs+jE7/unGKpA/ZxAxXsvQzNMHFpT3+/6S3Q=,76b343249331e4bbbfeb2c8fa67d51f1,2016-04-27,00003021501,000030215,28,28.0,None,25210.0,...,0.0,20954.0,20954.0,28,282.0,28.0,22228.0,20954.0,22315.0,20954.0
383,000030215,dSo/VEaThA95jXy3VHv+z54ZpKbjD58FdEcWhXvn26E=,cac36747da11fb2011e5fe649def9f15,2016-05-17,00003021501,000030215,28,28.0,None,NaN,...,NaN,21576.0,21576.0,28,282.0,28.0,22228.0,21576.0,22315.0,21576.0
384,000030215,oCAGCO+5pAhbqndOIFobR8GZywMpChZWG8IW4yyaEj4=,f1aa78d8e029e4c577d8c23579873334,2016-06-08,00003021501,000030215,28,28.0,EA,25205.0,...,0.0,20954.0,20954.0,28,282.0,28.0,22228.0,20954.0,22315.0,20954.0


,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PHARMACY_SUBMITTED_COST,...,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_99,QUANTITY_CLEAN,PLAN_PAY_99,PLAN_PAY_CLEAN,TOTAL_COST_99,TOTAL_COST_CLEAN
0,000030215,ZE1tW75yB2uQJTeDoGGVdEgk13jwPobZ66L7BQwuy1s=,2128c458187afdf637749d3eb97f2a17,2016-05-20,00003021501,000030215,28,282.0,None,None,...,None,1039.0,1039.0,28,282.0,282.0,22228.0,1039.0,22315.0,1039.0


CPU times: user 567 ms, sys: 5.85 ms, total: 573 ms
Wall time: 893 ms


In [105]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select left(ndc, 9) as ndc9, *
    from {unit_price}
    where ndc = '55513028310'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where ndc = '55513028310'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where encounter_key = 'fca62d854ab6d9f734edc6b36d3e94c0'
    ;
"""
df = read_sql(sql)
display(df)

,NDC9,NDC,N_CLAIM,START_DATE,END_DATE,UNIT_PRICE_FINAL,MEDIAN_TOTAL_COST_PER_UNIT,MEDIAN_PLAN_PAY_PER_UNIT,NADAC_PER_UNIT,ASP_PER_UNIT
0,555130283,55513028310,971961,1970-01-01,2030-12-31,147.5,158.5,147.5,None,8.222


,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PHARMACY_SUBMITTED_COST,...,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_99,QUANTITY_CLEAN,PLAN_PAY_99,PLAN_PAY_CLEAN,TOTAL_COST_99,TOTAL_COST_CLEAN
0,555130283,FOP+R2QF8GEE37DDpXqAXI1z6TKy3qsDPoDQKa6q4zY=,1e8dca71851f28b65b1c4fec8eca9a6a,2014-11-23,55513028310,555130283,1,60.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,60.0,2661.0,NaN,2713.0,NaN
1,555130283,h+PmwABfIQe2AG46BiX1XiJYs51ONfND9n6oz43DJDY=,4a4e823ffa0eabd0819e99e9ce682ec6,2016-05-21,55513028310,555130283,1,30.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,30.0,2661.0,NaN,2713.0,NaN
2,555130283,aMZKAsgHUrAAyjq35gqWwLbp5x3BH1OX2Z7KZdsY0fI=,351d2ed86348d4c3a9f9f72e168a8a5c,2016-10-26,55513028310,555130283,1,32.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,32.0,2661.0,NaN,2713.0,NaN
3,555130283,Dt5x/FivHm9pizk+Zg69e1xilI8xVQYtlTeAyQEL4kc=,9b4c8956a3337c4a121cda95bc457147,2019-03-04,55513028310,555130283,1,1655.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,1655.0,2661.0,NaN,2713.0,NaN
4,555130283,B3zvx0HPp8Vgs0wx7VGodNpqEUX4FPwpBccOkzUmL18=,1ea4959a38dbbc1ec79b43d6583a3a48,2019-10-08,55513028310,555130283,1,0.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,NaN,2661.0,NaN,2713.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,555130283,6l319f7HvuFiUeNAzY1CSK2Tx1d+5TyvtE0cvN2iQzs=,f9ebaf1332538e4a474a49b7ad2ac06a,2015-12-31,55513028310,555130283,1,10.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,10.0,2661.0,NaN,2713.0,NaN
680,555130283,FSjCsL09SKYs953sLyM9WzKCInpH2fPx5twYNumAH4M=,89a54cf4c48d729bd18fdd15f792313f,2016-03-07,55513028310,555130283,1,900.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,900.0,2661.0,NaN,2713.0,NaN
681,555130283,Rnor1MReMpyYe5Tlqm9oA0XxL/An9rIvvBezNchN3Do=,98031d76f9a50009c9df8d4bdbee6bc8,2016-03-30,55513028310,555130283,1,20.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,20.0,2661.0,NaN,2713.0,NaN
682,555130283,PEY2t1Fpuij0S9UFACCOKe2wZdZo2rQWjRwxylpD7ps=,0d5f456bce7bc88996a706aae57ddaee,2016-08-26,55513028310,555130283,1,50.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,50.0,2661.0,NaN,2713.0,NaN


,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PHARMACY_SUBMITTED_COST,...,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_99,QUANTITY_CLEAN,PLAN_PAY_99,PLAN_PAY_CLEAN,TOTAL_COST_99,TOTAL_COST_CLEAN
0,555130283,5WjGG+H/qei9Z64YAZhyc6jpuoyGiuBkCiTeZnLYBqA=,fca62d854ab6d9f734edc6b36d3e94c0,2016-03-02,55513028310,555130283,1,520.0,None,None,...,None,None,None,1,5000.0,520.0,2661.0,None,2713.0,None


CPU times: user 574 ms, sys: 0 ns, total: 574 ms
Wall time: 1.78 s


In [111]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select left(ndc, 9) as ndc9, *
    from {unit_price}
    where ndc = '00562780505'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where ndc = '00562780505'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where encounter_key = '294858bbbf9d9702d4c5b699fb6b35df'
    ;
"""
df = read_sql(sql)
display(df)

,NDC9,NDC,N_CLAIM,START_DATE,END_DATE,UNIT_PRICE_FINAL,MEDIAN_TOTAL_COST_PER_UNIT,MEDIAN_PLAN_PAY_PER_UNIT,NADAC_PER_UNIT,ASP_PER_UNIT
0,005627805,00562780505,75514,1970-01-01,2030-12-31,105.0,109.0,105.0,None,78.989


,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PHARMACY_SUBMITTED_COST,...,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_99,QUANTITY_CLEAN,PLAN_PAY_99,PLAN_PAY_CLEAN,TOTAL_COST_99,TOTAL_COST_CLEAN
0,005627805,vq4HvyOmvaVUlFPJLhisvn9x6LkuvmjQBR2+EGhtWxk=,8b82623f2978c566e1430a7699cad64d,2018-04-19,00562780505,005627805,30,1.0,None,NaN,...,NaN,NaN,NaN,30,1500.0,1.0,133.0,NaN,206.15,NaN
1,005627805,UYYb2X6md3zLnJiuRKM33EUWD/8PyQa0EPBmEqI5RAs=,4a172c5d5dc4886e429925f10b7168f9,2021-07-14,00562780505,005627805,1,1.0,None,NaN,...,NaN,NaN,NaN,1,1500.0,1.0,133.0,NaN,206.15,NaN
2,005627805,GxxI89iVkIEriFXwoLJLOf5nigsJjsqjkEFwnZa157c=,fc10ca78ecbb4d8ba774ecc649264a48,2021-12-29,00562780505,005627805,1,1.0,EA,160.0,...,0.0,118.0,118.0,1,1500.0,1.0,133.0,118.0,206.15,118.0
3,005627805,X2GcdBqfpBApgIWmIRBX++J64BlXOY9bdIEIWu9zRZo=,b00ae05bff0547061c66239232eeacea,2017-07-06,00562780505,005627805,1,1500.0,None,NaN,...,NaN,NaN,NaN,1,1500.0,1500.0,133.0,NaN,206.15,NaN
4,005627805,2q1uYKBR6pzm0Z+gmU/qVKyk+v6nrZj7RRpGSIEuJI4=,207792ed5c64d461c5fb14d110d4a2f7,2018-03-12,00562780505,005627805,28,1.0,None,110.0,...,109.0,0.0,109.0,28,1500.0,1.0,133.0,NaN,206.15,109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,005627805,vAj3KlhHBQuJmkVGS/xa3ZBZrw/5FJYlOjuwGTekv4A=,c8a99f596aa795885a3f965416d363be,2016-09-08,00562780505,005627805,1,1.0,None,NaN,...,NaN,0.0,0.0,1,1500.0,1.0,133.0,NaN,206.15,NaN
214,005627805,4SsN1yOC4nnKoJUD7bz/DOtf4vLYyDNOTslBqrEsNfg=,946a49801960913ecb81c77b85595821,2018-03-16,00562780505,005627805,1,1500.0,None,NaN,...,NaN,NaN,NaN,1,1500.0,1500.0,133.0,NaN,206.15,NaN
215,005627805,Pb7Cji3mCN+fbpvpyJMxW8xdJSuhJNAq9LGmZlDmk20=,d679b28e3b22eddf51553c4428b3aea1,2021-07-13,00562780505,005627805,1,1500.0,None,NaN,...,NaN,NaN,NaN,1,1500.0,1500.0,133.0,NaN,206.15,NaN
216,005627805,AON0kXuXkr2Cc2EgWLDaFz6nd32u6s9Uxk9mmqw/N5I=,9d71efd4fbbb5d71b6739f4f93abd3ef,2021-10-18,00562780505,005627805,1,1.0,EA,160.0,...,0.0,118.0,118.0,1,1500.0,1.0,133.0,118.0,206.15,118.0


,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PHARMACY_SUBMITTED_COST,...,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_99,QUANTITY_CLEAN,PLAN_PAY_99,PLAN_PAY_CLEAN,TOTAL_COST_99,TOTAL_COST_CLEAN
0,003782046,OB4c7wXvbiBTM4PFXWuYFvGbfiD7w5rMMqw/g7wvFU0=,294858bbbf9d9702d4c5b699fb6b35df,2020-02-07,00378204601,003782046,30,180000.0,None,569.0,...,4.0,None,4.0,30,360000.0,180000.0,1049.25,None,951.0,4.0


CPU times: user 520 ms, sys: 0 ns, total: 520 ms
Wall time: 23 s
